In [20]:
from sklearn.ensemble import BaggingClassifier,VotingClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

from statistics import multimode
import copy
import pandas as pd
import numpy as np
import random

#### load data

In [21]:
#Read ds8004_hw3_q2_data.csv file and split it to train,test

data=pd.read_csv('ds8004_hw3_q2_data.csv')
train,test=train_test_split(data, test_size=100, random_state=0, shuffle=True, stratify=data['target'])

# specified all colomns until last one for X_train and the last one for the Y_train
X,y=train[train.columns[:-1]].values,train['target'].values

# specified all colomns until last one for X_test and the last one for the Y_test
X_test,y_test=test[test.columns[:-1]].values,test['target'].values

## 2a

##### base_estimator

In [22]:
# Defined the clasifier which is DecisionTree. I found 'gini'works better.

estimator=DecisionTreeClassifier(criterion='gini', splitter='best')

##### sklearn BaggingClassifier

In [23]:
#Implement our model(estimator) using sklearn library function BaggingClassifier.
sklearn_clf = BaggingClassifier(base_estimator=estimator,n_estimators=10, random_state=0).fit(X, y)

y_P=sklearn_clf.predict(X_test)
# Displayed the performance of the Sklearn BagginClassifier 

print(classification_report(y_test,y_P))
print(confusion_matrix(y_test,y_P))

              precision    recall  f1-score   support

           1       0.58      0.75      0.65        24
           2       0.60      0.58      0.59        26
           3       0.77      0.80      0.78        25
           4       0.94      0.68      0.79        25

    accuracy                           0.70       100
   macro avg       0.72      0.70      0.70       100
weighted avg       0.72      0.70      0.70       100

[[18  6  0  0]
 [ 9 15  2  0]
 [ 2  2 20  1]
 [ 2  2  4 17]]


#### Our Impliment for BaggingClassifier

In [24]:
#Defined the Bagging_Clf class which construct the list of
#estimator regard of the n_estimators
# In -init- we made copy from base-estimator & stored in the list
# We used deepcopy because the base_estimator do not have effect on each other

class Bagging_Clf:
    def __init__(self, base_estimator,n_estimators=10):
        self.n_estimators = n_estimators
        self.estimators=[]
        for i in range(self.n_estimators):
            self.estimators.append(copy.deepcopy(base_estimator))
            
#We want to create 10 different data sets for our 10 estimator
#Here we used random.choice function to create n numberes b/W 1 to n
#We create b_ind then we append it to X
# We will repeat this action for all estimator in order to create dataset for them
    
    def data_replacment(self,X,y):
        index=np.arange(len(X))
        b_X=[]
        b_y=[]
        for i in range(self.n_estimators):
            b_ind=np.random.choice(index,len(X), replace=True)
            b_X.append(X[b_ind])
            b_y.append(y[b_ind])
        return b_X,b_y
    
#Here we fit every dataset for each estimator
    
    def fit(self,X,y):
        b_X, b_y=self.data_replacment(X,y)
        for i in range(self.n_estimators):
            self.estimators[i].fit(b_X[i],b_y[i])
        return self
#Here we used for loop to call each estimator to pass them to the predict function
#predict(X) and store their predicts in Y list
# lets say our #of estimators are 10 so Y is a list which contains 10 prediction list
        
    def predict(self,X):
        Y=[]
        for i in range(self.n_estimators):
            Y.append(self.estimators[i].predict(X).tolist())
        y_pre=[]
        Y_pre=[]
        for i in range(len(X)):
# y_i contains ys in number of our estimators
            y_i=[y[i] for y in Y]
    
# stored all predictions in Y_pre from each estimators that here means y_i
# Then we need to find their mode

            Y_pre.append(y_i)
#multimode return the y predict which have the highest mode compare with others
#random.choice :Lets assume if two modes retured so random.choice will choice it 

            y_pre.append(np.random.choice(multimode(y_i)))
        return y_pre,Y_pre,Y

In [25]:

#
my_clf=Bagging_Clf(base_estimator=estimator,n_estimators=10).fit(X, y)


In [26]:
y_Pre,Y_Pre,Y=my_clf.predict(X_test)
print(classification_report(y_test,y_Pre))
print(confusion_matrix(y_test,y_Pre))

              precision    recall  f1-score   support

           1       0.68      0.71      0.69        24
           2       0.68      0.73      0.70        26
           3       0.71      0.68      0.69        25
           4       0.74      0.68      0.71        25

    accuracy                           0.70       100
   macro avg       0.70      0.70      0.70       100
weighted avg       0.70      0.70      0.70       100

[[17  6  0  1]
 [ 6 19  1  0]
 [ 1  2 17  5]
 [ 1  1  6 17]]


In [29]:
print(Y)

[[3, 3, 4, 4, 1, 1, 2, 2, 4, 4, 1, 2, 1, 2, 1, 2, 2, 3, 3, 4, 3, 4, 3, 2, 4, 3, 3, 4, 3, 3, 1, 1, 4, 2, 3, 3, 3, 2, 4, 1, 1, 2, 2, 1, 2, 2, 4, 4, 2, 1, 1, 4, 4, 4, 1, 2, 1, 3, 4, 3, 3, 2, 2, 1, 3, 1, 3, 1, 2, 2, 4, 4, 2, 2, 1, 2, 3, 2, 4, 4, 2, 2, 2, 1, 4, 3, 4, 3, 4, 2, 4, 2, 3, 2, 3, 1, 4, 1, 1, 4], [3, 3, 3, 3, 4, 1, 2, 2, 1, 4, 3, 3, 2, 2, 1, 2, 1, 4, 3, 4, 3, 4, 2, 1, 4, 4, 3, 3, 4, 3, 1, 1, 4, 2, 3, 4, 3, 1, 3, 1, 1, 1, 3, 1, 1, 3, 4, 1, 2, 2, 2, 1, 4, 3, 1, 2, 2, 4, 4, 2, 3, 2, 2, 2, 3, 1, 2, 3, 2, 1, 2, 3, 2, 4, 1, 2, 3, 2, 3, 4, 2, 2, 2, 2, 4, 4, 4, 4, 1, 1, 4, 4, 4, 3, 1, 4, 4, 1, 1, 4], [3, 3, 2, 4, 2, 1, 3, 2, 3, 4, 1, 4, 2, 1, 1, 2, 1, 2, 3, 4, 3, 3, 2, 1, 4, 4, 3, 2, 4, 3, 1, 1, 4, 1, 3, 4, 3, 2, 2, 2, 1, 2, 2, 1, 3, 3, 4, 2, 2, 3, 2, 3, 4, 2, 3, 2, 2, 4, 4, 1, 2, 1, 2, 1, 4, 1, 3, 2, 3, 1, 2, 4, 2, 4, 1, 2, 3, 2, 2, 3, 2, 2, 2, 2, 2, 1, 4, 4, 4, 2, 4, 2, 4, 2, 3, 2, 4, 1, 2, 4], [1, 3, 3, 4, 1, 1, 3, 2, 3, 2, 1, 3, 2, 1, 1, 2, 1, 2, 2, 4, 3, 4, 2, 1, 3, 4, 3, 3, 3, 3, 1,

In [28]:
Y_Pre

[[3, 3, 3, 1, 3, 3, 3, 3, 3, 3],
 [3, 3, 3, 3, 3, 4, 4, 3, 3, 3],
 [4, 3, 2, 3, 1, 4, 4, 3, 3, 3],
 [4, 3, 4, 4, 3, 4, 4, 4, 3, 4],
 [1, 4, 2, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [2, 2, 3, 3, 1, 2, 3, 2, 1, 4],
 [2, 2, 2, 2, 2, 2, 2, 1, 2, 2],
 [4, 1, 3, 3, 3, 3, 3, 3, 3, 1],
 [4, 4, 4, 2, 4, 4, 3, 4, 4, 4],
 [1, 3, 1, 1, 1, 1, 1, 1, 1, 1],
 [2, 3, 4, 3, 4, 4, 4, 4, 4, 4],
 [1, 2, 2, 2, 2, 2, 2, 2, 2, 2],
 [2, 2, 1, 1, 2, 2, 2, 1, 2, 2],
 [1, 1, 1, 1, 1, 1, 3, 1, 1, 2],
 [2, 2, 2, 2, 3, 1, 2, 1, 2, 2],
 [2, 1, 1, 1, 3, 1, 1, 1, 1, 1],
 [3, 4, 2, 2, 1, 4, 4, 3, 2, 2],
 [3, 3, 3, 2, 3, 3, 2, 3, 3, 1],
 [4, 4, 4, 4, 3, 3, 1, 4, 4, 3],
 [3, 3, 3, 3, 4, 3, 3, 3, 3, 3],
 [4, 4, 3, 4, 4, 4, 3, 4, 2, 3],
 [3, 2, 2, 2, 2, 2, 2, 2, 2, 2],
 [2, 1, 1, 1, 2, 2, 1, 2, 1, 2],
 [4, 4, 4, 3, 4, 3, 3, 4, 4, 4],
 [3, 4, 4, 4, 4, 3, 1, 3, 4, 4],
 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
 [4, 3, 2, 3, 2, 3, 4, 3, 3, 2],
 [3, 4, 4, 3, 4, 3, 3, 4, 3, 4],
 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
 [1, 1, 1,

In [32]:
print(y_Pre)

[3, 3, 3, 4, 1, 1, 2, 2, 3, 4, 1, 4, 2, 2, 1, 2, 1, 2, 3, 4, 3, 4, 2, 2, 4, 4, 3, 3, 3, 3, 1, 1, 4, 2, 3, 4, 3, 1, 3, 1, 1, 2, 2, 1, 2, 3, 4, 1, 2, 1, 2, 3, 4, 4, 1, 2, 2, 3, 4, 2, 3, 2, 2, 1, 3, 1, 3, 1, 3, 1, 1, 3, 2, 4, 1, 2, 3, 2, 4, 4, 2, 2, 2, 2, 4, 1, 4, 4, 4, 1, 4, 2, 3, 3, 2, 1, 4, 1, 1, 4]


In [17]:
estimators_acc=[accuracy_score(y_test,y) for y in Y]
print('Individual estimators accuracy:',estimators_acc)
print(f'mean accuracy:{np.round(np.mean(estimators_acc),3)} (+/- {np.round(np.std(estimators_acc),3)})')
print('Bagging accuracy:',accuracy_score(y_test,y_Pre))

Individual estimators accuracy: [0.66, 0.59, 0.58, 0.53, 0.58, 0.68, 0.68, 0.64, 0.66, 0.6]
mean accuracy:0.62 (+/- 0.048)
Bagging accuracy: 0.75


## 2b

##### estimators

In [18]:
#We defined our classifier as DecisionTree,SVC and GaussianNB
# append all classifiers in estimator list
estimators=[]
estimator1=DecisionTreeClassifier(criterion='gini', splitter='best')
estimators.append(('DT',estimator1))
estimator2=SVC(C=1.0, kernel='rbf', gamma='scale')
estimators.append(('SVM',estimator2))
estimator3=GaussianNB()
estimators.append(('GNB',estimator3))

##### sklearn VotingClassifier

In [11]:
#We used Voting classifier from sklearn library
#We stored our all y predicts for each classifies on X-test
sklearn_clf = VotingClassifier(estimators=estimators).fit(X, y)
y_P=sklearn_clf.predict(X_test)

# Visually report the performance of the sklearn classifiers and VotingClassifier
print(classification_report(y_test,y_P))
print(confusion_matrix(y_test,y_P))

              precision    recall  f1-score   support

           1       0.47      0.58      0.52        24
           2       0.48      0.46      0.47        26
           3       0.64      0.56      0.60        25
           4       0.70      0.64      0.67        25

    accuracy                           0.56       100
   macro avg       0.57      0.56      0.56       100
weighted avg       0.57      0.56      0.56       100

[[14  8  1  1]
 [12 12  2  0]
 [ 3  2 14  6]
 [ 1  3  5 16]]


#### Our Implimentation of VotingClassifier

In [19]:
#

class Voting_Clf:
    def __init__(self, estimators): 
        self.estimators=estimators
        self.n_estimators =len(estimators)
#we add this add_estimator method in case if 
# latar want to add more estimator

    def add_estimator(self, new_estimator):
        self.estimators.append(new_estimator)
        self.n_estimators +=1
        
    def fit(self,X,y):
        for i in range(self.n_estimators):
            self.estimators[i][1].fit(X,y)
        return self
        
    def predict(self,X):
        Y=[]
        for i in range(self.n_estimators):
            Y.append(self.estimators[i][1].predict(X).tolist())
        y_pre=[]
        Y_pre=[]
        for i in range(len(X)):
            y_i=[y[i] for y in Y]
            Y_pre.append(y_i)
            y_pre.append(np.random.choice(multimode(y_i)))
        return y_pre,Y_pre,Y

In [13]:
my_clf=Voting_Clf(estimators=estimators).fit(X, y)
y_Pre,Y_Pre,Y=my_clf.predict(X_test)
print(classification_report(y_test,y_Pre))
print(confusion_matrix(y_test,y_Pre))

              precision    recall  f1-score   support

           1       0.52      0.54      0.53        24
           2       0.58      0.54      0.56        26
           3       0.62      0.64      0.63        25
           4       0.72      0.72      0.72        25

    accuracy                           0.61       100
   macro avg       0.61      0.61      0.61       100
weighted avg       0.61      0.61      0.61       100

[[13  8  2  1]
 [ 9 14  3  0]
 [ 2  1 16  6]
 [ 1  1  5 18]]


In [14]:
estimators_acc=[accuracy_score(y_test,y) for y in Y]
print('estimators method:',[estimator[0] for estimator in estimators])
print('Individual estimators accuracy:',estimators_acc)
print(f'mean accuracy:{np.round(np.mean(estimators_acc),3)} (+/- {np.round(np.std(estimators_acc),3)})')
print('Voting accuracy:',accuracy_score(y_test,y_Pre))

estimators method: ['DT', 'SVM', 'GNB']
individual estimators accuracy: [0.73, 0.41, 0.48]
mean accuracy:0.54 (+/- 0.137)
Voting accuracy: 0.61
